In [ ]:
import numpy as np
import random

# Define the Problem
def distance_matrix(num_cities):
    # Random symmetric matrix of distances
    matrix = np.random.rand(num_cities, num_cities)
    matrix = (matrix + matrix.T) / 2  # Make it symmetric
    np.fill_diagonal(matrix, 0)  # Distance to self is 0
    return matrix

# Parameters
num_cities = 5
num_ants = num_cities
alpha = 1.0  # Pheromone importance
beta = 2.0   # Distance importance
evaporation_rate = 0.5
pheromone_constant = 100
num_iterations = 100

# Initialize distance and pheromone matrices
distances = distance_matrix(num_cities)
pheromones = np.ones((num_cities, num_cities))

# Initialize Ant Class
class Ant:
    def __init__(self, num_cities):
        self.num_cities = num_cities
        self.visited = []
        self.total_distance = 0

    def visit_city(self, city):
        if self.visited:
            last_city = self.visited[-1]
            self.total_distance += distances[last_city, city]
        self.visited.append(city)

    def calculate_tour_length(self):
        # Complete the tour by returning to the start city
        return self.total_distance + distances[self.visited[-1], self.visited[0]]

# Construct Solutions
def construct_solutions():
    ants = [Ant(num_cities) for _ in range(num_ants)]
    for ant in ants:
        start_city = random.randint(0, num_cities - 1)
        ant.visit_city(start_city)

        # For each ant, build a full tour
        for _ in range(num_cities - 1):
            current_city = ant.visited[-1]
            probabilities = []
            for next_city in range(num_cities):
                if next_city not in ant.visited:
                    pheromone = pheromones[current_city, next_city] ** alpha
                    visibility = (1.0 / distances[current_city, next_city]) ** beta
                    probabilities.append((next_city, pheromone * visibility))

            total_probability = sum(p[1] for p in probabilities)
            probabilities = [(city, prob / total_probability) for city, prob in probabilities]
            next_city = random.choices([city for city, prob in probabilities],
                                       weights=[prob for city, prob in probabilities])[0]
            ant.visit_city(next_city)

    return ants

# Update Pheromones
def update_pheromones(ants):
    global pheromones
    pheromones *= (1 - evaporation_rate)  # Evaporate pheromones

    for ant in ants:
        tour_length = ant.calculate_tour_length()
        for i in range(len(ant.visited) - 1):
            city_i, city_j = ant.visited[i], ant.visited[i + 1]
            pheromones[city_i, city_j] += pheromone_constant / tour_length
            pheromones[city_j, city_i] += pheromone_constant / tour_length  # Symmetric path

        # Complete the tour by adding pheromones on the path back to the start city
        pheromones[ant.visited[-1], ant.visited[0]] += pheromone_constant / tour_length
        pheromones[ant.visited[0], ant.visited[-1]] += pheromone_constant / tour_length

# ACO Algorithm
def ant_colony_optimization():
    best_tour = None
    best_distance = float('inf')

    for iteration in range(num_iterations):
        # Step 1: Construct solutions
        ants = construct_solutions()

        # Step 2: Evaluate solutions and find the best tour
        for ant in ants:
            tour_length = ant.calculate_tour_length()
            if tour_length < best_distance:
                best_distance = tour_length
                best_tour = ant.visited.copy()

        # Step 3: Update pheromones
        update_pheromones(ants)

        # Optional: Print the best solution in the current iteration
        print(f"Iteration {iteration + 1}: Best distance = {best_distance}, Tour = {best_tour}")

    return best_tour, best_distance

# Run ACO
best_tour, best_distance = ant_colony_optimization()
print(f"Best solution found: Distance = {best_distance}, Tour = {best_tour}")


Iteration 1: Best distance = 2.3468550877109764, Tour = [2, 3, 4, 1, 0]
Iteration 2: Best distance = 2.0744110953532684, Tour = [2, 0, 4, 1, 3]
Iteration 3: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 4: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 5: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 6: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 7: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 8: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 9: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 10: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 11: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 12: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 13: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]
Iteration 14: Best distance = 2.074411095353268, Tour = [4, 0, 2, 3, 1]

In [ ]:
import numpy as np
import random

# Define the TSP problem with a distance matrix
def create_distance_matrix(n_cities):
    np.random.seed(0)
    matrix = np.random.randint(1, 100, size=(n_cities, n_cities))
    np.fill_diagonal(matrix, 0)  # No distance between a city and itself
    return matrix

# Parameters
n_cities = 10                # Number of cities
n_ants = 20                  # Number of ants
n_iterations = 50           # Number of iterations
alpha = 1                    # Influence of pheromone
beta = 2                     # Influence of distance
evaporation_rate = 0.5       # Pheromone evaporation rate
initial_pheromone = 1        # Initial pheromone level

# Initialize distance and pheromone matrices
distance_matrix = create_distance_matrix(n_cities)
pheromone_matrix = np.ones((n_cities, n_cities)) * initial_pheromone

# Ant class representing a single ant in the colony
class Ant:
    def __init__(self, n_cities):
        self.n_cities = n_cities
        self.route = []
        self.distance_travelled = 0

    def select_next_city(self, current_city, visited):
        probabilities = []
        for city in range(self.n_cities):
            if city not in visited:
                pheromone = pheromone_matrix[current_city][city] ** alpha
                heuristic = (1 / distance_matrix[current_city][city]) ** beta
                probabilities.append(pheromone * heuristic)
            else:
                probabilities.append(0)

        probabilities = np.array(probabilities) / sum(probabilities)
        next_city = np.random.choice(range(self.n_cities), p=probabilities)
        return next_city

    def find_route(self):
        current_city = random.randint(0, self.n_cities - 1)
        self.route = [current_city]
        visited = set(self.route)

        while len(visited) < self.n_cities:
            next_city = self.select_next_city(current_city, visited)
            self.route.append(next_city)
            self.distance_travelled += distance_matrix[current_city][next_city]
            visited.add(next_city)
            current_city = next_city

        # Complete the tour by returning to the starting city
        self.distance_travelled += distance_matrix[self.route[-1]][self.route[0]]
        self.route.append(self.route[0])

# Update pheromone matrix based on the best route found
def update_pheromones(ants):
    global pheromone_matrix
    pheromone_matrix *= (1 - evaporation_rate)  # Apply evaporation

    for ant in ants:
        for i in range(len(ant.route) - 1):
            city_from = ant.route[i]
            city_to = ant.route[i + 1]
            pheromone_matrix[city_from][city_to] += 1.0 / ant.distance_travelled
            pheromone_matrix[city_to][city_from] += 1.0 / ant.distance_travelled

# Ant Colony Optimization
def ant_colony_optimization():
    best_route = None
    best_distance = float('inf')

    for iteration in range(n_iterations):
        ants = [Ant(n_cities) for _ in range(n_ants)]

        for ant in ants:
            ant.find_route()
            if ant.distance_travelled < best_distance:
                best_distance = ant.distance_travelled
                best_route = ant.route

        update_pheromones(ants)
        print(f"Iteration {iteration + 1}: Best distance = {best_distance}")

    return best_route, best_distance

# Run ACO for TSP
best_route, best_distance = ant_colony_optimization()
print(f"Best route found: {best_route} with distance: {best_distance}")

Iteration 1: Best distance = 139
Iteration 2: Best distance = 130
Iteration 3: Best distance = 130
Iteration 4: Best distance = 130
Iteration 5: Best distance = 130
Iteration 6: Best distance = 130
Iteration 7: Best distance = 130
Iteration 8: Best distance = 130
Iteration 9: Best distance = 130
Iteration 10: Best distance = 130
Iteration 11: Best distance = 130
Iteration 12: Best distance = 130
Iteration 13: Best distance = 130
Iteration 14: Best distance = 130
Iteration 15: Best distance = 130
Iteration 16: Best distance = 130
Iteration 17: Best distance = 130
Iteration 18: Best distance = 130
Iteration 19: Best distance = 130
Iteration 20: Best distance = 130
Iteration 21: Best distance = 130
Iteration 22: Best distance = 130
Iteration 23: Best distance = 130
Iteration 24: Best distance = 130
Iteration 25: Best distance = 118
Iteration 26: Best distance = 118
Iteration 27: Best distance = 118
Iteration 28: Best distance = 118
Iteration 29: Best distance = 118
Iteration 30: Best dist